# Exercise 1
### Amandus Omholt Nygaard, Yawar Mahmood, Hirut Gebremedhin

## Problem 1

a)

$ K_2(A) = ||A||_2 ||A^{-1}||_2 $
It is given from the lecture notes that $||A||_2 = \sqrt{\lambda_{max}} $, where $\lambda_{max}$ is the eigenvalue of $A^TA$. We can use this result to find $||A^{-1}||_2$, which is given by:

$||A^{-1}||_2 = \sqrt{\Lambda_{max}}$, where $\Lambda_{max}$ is the eigenvalue of $(A^TA)^{-1}$

Since $(A^TA)^{-1} = (A^{-1})^TA^{-1}$, we can deduce that the eigenvalues of $(A^TA)^{-1}$ are the inverse of $A^TA$. Therefore:

$ \Lambda_{max} = \frac{1}{\lambda_{min}} $, where $\lambda_{min}$ is the eigenvalue of $A^TA$.

The condition number is therefore given by:
$$ K_2(A) = ||A||_2 ||A^{-1}||_2 = \sqrt{\frac{\lambda_{max}}{\lambda_{min}}} $$

b)

Given an orthogonal matrix Q: $Q^T = Q^{-1}$

Therefore:  $ K_2(Q) = ||Q||_2 ||Q^{-1}||_2 = ||Q||_2 ||Q^{T}||_2 $ 

Further: $QQ^{-1} = QQ^T = I$, and it is known that the eigenvalues of I are all 1 (by construction)

The condition number is therefore given by:
$$ K_2(Q) = ||Q||_2 ||Q^{-1}||_2 = \sqrt{\frac{\lambda_{max}}{\lambda_{min}}} = 1 $$
Where $ \lambda_{max} \land \lambda_{min} $ are the eigenvalues of I. 

c)

It is given that:
$$  K_2(A) = \sqrt{\frac{\lambda_{max}}{\lambda_{min}}} = 1 $$
Which implies that $ \lambda_{max} = \lambda_{min} $

If the bigget and smallest eigenvalues are equal to each other, then all the eigenvalues has to be equal to each other. 

## Problem 2

In [1]:
import numpy as np
import scipy.linalg as sp
import time

In [2]:
def mylu(A):
    LU = np.copy(A)
    n = len(LU)
    p = np.arange(n)
    for k in range(n):
        max_index = np.argmax(np.abs(LU[p[k:],k]))
        p[k],p[max_index+k] = p[max_index+k],p[k]
        
        for i in range(k+1,n):
            LU[p[i],k] = LU[p[i],k]/LU[p[k],k]
            
            for j in range(k+1,n):
                LU[p[i],j] = LU[p[i],j] - LU[p[i],k]*LU[p[k],j]
    
    return LU,p

In [3]:
A = np.array ([[2.0, 5, 8, 7], [5, 2, 2, 8], [7, 5, 6, 6], [5, 4, 4, 8]])

In [4]:
def mylutest(A):
    A_lu,p_lu = mylu(A)
    print("mylu:")
    print(A_lu[p_lu[:],:])
    print()
    
    P,L,U = sp.lu(A)
    print("Scipy lu:")
    print("U=",U)
    print()
    print("L=",L)

In [5]:
mylutest(A)

mylu:
[[ 7.          5.          6.          6.        ]
 [ 0.28571429  3.57142857  6.28571429  5.28571429]
 [ 0.71428571  0.12       -1.04        3.08      ]
 [ 0.71428571 -0.44       -0.46153846  7.46153846]]

Scipy lu:
U= [[ 7.          5.          6.          6.        ]
 [ 0.          3.57142857  6.28571429  5.28571429]
 [ 0.          0.         -1.04        3.08      ]
 [ 0.          0.          0.          7.46153846]]

L= [[ 1.          0.          0.          0.        ]
 [ 0.28571429  1.          0.          0.        ]
 [ 0.71428571  0.12        1.          0.        ]
 [ 0.71428571 -0.44       -0.46153846  1.        ]]


In [6]:
A = np.array([[1,2,3],[1,2,6],[1,2,9]])

A_lu,p_lu = mylu(A)
print(A_lu[p_lu[:],:])

/tmp/ipykernel_96231/1693290250.py:10: RuntimeWarning: invalid value encountered in long_scalars
  LU[p[i],k] = LU[p[i],k]/LU[p[k],k]


ValueError: cannot convert float NaN to integer

Get divide by zero error when the matrix is not of full rank.

## Problem 3

In [7]:
def forward_substitution(A,b):
    n = len(b)
    x = np.zeros(n)
    x[0] = b[0]/A[0,0]
    for i in range(1,n):
        x[i] = (b[i] - sum(np.multiply(x[0:i],A[i,0:i])))/A[i,i]
    return x

def backward_substitution(A,b):
    n = len(b)
    x = np.zeros(n)
    x[n-1] = b[n-1]/A[n-1,n-1]
    for i in range(n-2,-1,-1):
        x[i] = (b[i] - sum(np.multiply(x[i+1:n],A[i,i+1:n])))/A[i,i]
    return x

In [8]:
def my_solve(A,b):
    LU,p = mylu(A)
    
    LU = LU[p[:],:]
    
    y = forward_substitution((LU-np.diag(np.diagonal(LU)))+np.diag(np.ones(len(LU))),b[p[:]])
    
    x = backward_substitution(LU,y)
    
    return x

In [9]:
m = 200
n = 100

L = np.random.rand(n,n)*10
L = np.tril(L)-np.diag(np.diagonal(L))+np.diag(np.ones(n))

U = np.triu(np.random.rand(n,n)*10)

A = np.matmul(L,U)

b = np.random.rand(len(A),m)

#test 1
time_1 = time.time()
for i in range(m):
    x = my_solve(A,b[:,i])
time_1 = time.time() - time_1

#test 2
time_2 = time.time()
A,p = mylu(A)
    
A = A[p[:],:]

for i in range(m):
    x = forward_substitution((A-np.diag(np.diagonal(A)))+np.diag(np.ones(len(A))),b[:,i])
    
    x = backward_substitution(A,x)
    
time_2 = time.time() - time_2

#test 3
time_3 = time.time()
for i in range(m):
    x = sp.solve(A,b[:,i])
time_3 = time.time() - time_3

print("LU decomp {m} times: ",time_1)
print("LU decomp 1 time",time_2)
print("Scipy: ",time_3)

LU decomp {m} times:  43.20029544830322
LU decomp 1 time 0.40660619735717773
Scipy:  0.07245922088623047
